<a href="https://colab.research.google.com/github/hoangtung386/Kaggle_notebook/blob/main/Segment_train_RSNA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

rsna_2023_abdominal_trauma_detection_path = kagglehub.competition_download('rsna-2023-abdominal-trauma-detection')

print('Data source import complete.')


# Install packages

In [ ]:
!pip install -q monai wandb nibabel torchsummary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 57.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 49.9 MB/s eta 0:00:0000:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.8 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependenc

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from matplotlib.colors import ListedColormap
from sklearn.metrics import confusion_matrix
from monai.transforms import (Compose, LoadImaged, EnsureChannelFirstd, Spacingd, ScaleIntensityRanged,
                              Orientationd, CropForegroundd, SpatialPadd, RandCropByPosNegLabeld,
                              RandAffined, ToTensord, RandSpatialCropd, ResizeWithPadOrCropd)
from monai.metrics import DiceMetric
from monai.losses import DiceLoss
from monai.data import CacheDataset, list_data_collate
from monai.inferers import sliding_window_inference
from monai.utils import set_determinism
from tqdm import tqdm
from scipy.spatial.distance import directed_hausdorff
import wandb
from datetime import datetime

<frozen importlib._bootstrap_external>:1241: FutureWarning: The cuda.cudart module is deprecated and will be removed in a future release, please switch to use the cuda.bindings.runtime module instead.
2025-10-25 08:54:19.623377: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761382459.848325      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761382459.914859      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it w

# W&B Authentication

In [ ]:
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
use_wandb = True  # Đặt thành False nếu không muốn dùng wandb

In [ ]:
if use_wandb:
    try:
        wandb_api = user_secrets.get_secret("WANDB_API_KEY")
        wandb.login(key=wandb_api)
        print("W&B authentication successful!")
    except Exception as e:
        print(f"W&B authentication failed: {e}")
        print("Continuing without W&B logging")
        use_wandb = False

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hoangtung386 (levuhoangtung) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


W&B authentication successful!


In [ ]:
# Set seed
random_seed = 42
set_determinism(seed=random_seed)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

# I . Data preprocessing

## 1. Paths và Config

In [ ]:
base_path = '/kaggle/input/rsna-2023-abdominal-trauma-detection/'
output_dir = '/kaggle/working'
os.makedirs(output_dir, exist_ok=True)

CONFIG = {
    'seed': random_seed,
    'train_split': 0.8,
    'batch_size': 1,
    'num_epochs': 20,
    'learning_rate': 1e-3,
    'spatial_size': (96, 96, 96),
    'init_features': 16,
    'num_classes': 6,
    'cache_rate': 0,
    'num_workers': 2,
}

## 2. Load Data

In [ ]:
series_meta = pd.read_csv(os.path.join(base_path, 'train_series_meta.csv'))
labels = pd.read_csv(os.path.join(base_path, 'train_2024.csv'))

seg_path = os.path.join(base_path, 'segmentations')
seg_files = [f for f in os.listdir(seg_path) if f.endswith('.nii')]

In [ ]:
data_list = []
for seg in seg_files:
    series_id = int(seg.split('.')[0])
    patient_row = series_meta[series_meta['series_id'] == series_id]
    if not patient_row.empty:
        patient_id = int(patient_row['patient_id'].values[0])
        image_dir = os.path.join(base_path, f'train_images/{patient_id}/{series_id}')
        seg_file = os.path.join(seg_path, seg)
        data_list.append({
            'image': image_dir,
            'seg': seg_file
        })

In [ ]:
train_list, val_list = train_test_split(
    data_list,
    test_size=1-CONFIG['train_split'],
    random_state=CONFIG['seed'],
    shuffle=True
)
print(f"Train samples: {len(train_list)}, Val samples: {len(val_list)}")

Train samples: 164, Val samples: 42


## 3. Transforms

In [ ]:
train_transforms = Compose([
    LoadImaged(keys=['image', 'seg']),
    EnsureChannelFirstd(keys=['image', 'seg']),
    Orientationd(keys=['image', 'seg'], axcodes='RAS'),
    Spacingd(keys=['image', 'seg'], pixdim=(1.0, 1.0, 1.0), mode=('bilinear', 'nearest')),
    ScaleIntensityRanged(keys=['image'], a_min=-125, a_max=275, b_min=0.0, b_max=1.0, clip=True),
    CropForegroundd(keys=['image', 'seg'], source_key='image'),
    SpatialPadd(keys=['image', 'seg'], spatial_size=CONFIG['spatial_size']),
    RandCropByPosNegLabeld(keys=['image', 'seg'], label_key='seg',
                           spatial_size=CONFIG['spatial_size'],
                           pos=1.0, neg=1.0, num_samples=2),
    RandAffined(keys=['image', 'seg'], prob=0.2, rotate_range=0.1,
                scale_range=0.1, mode=('bilinear', 'nearest')),
    ToTensord(keys=['image', 'seg'])
])

/usr/local/lib/python3.11/dist-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.spatial.dictionary Orientationd.__init__:labels: Current default value of argument `labels=(('L', 'R'), ('P', 'A'), ('I', 'S'))` was changed in version None from `labels=(('L', 'R'), ('P', 'A'), ('I', 'S'))` to `labels=None`. Default value changed to None meaning that the transform now uses the 'space' of a meta-tensor, if applicable, to determine appropriate axis labels.
  warn_deprecated(argname, msg, warning_category)


In [ ]:
val_transforms = Compose([
    LoadImaged(keys=['image', 'seg']),
    EnsureChannelFirstd(keys=['image', 'seg']),
    Orientationd(keys=['image', 'seg'], axcodes='RAS'),
    Spacingd(keys=['image', 'seg'], pixdim=(1.0, 1.0, 1.0), mode=('bilinear', 'nearest')),
    ScaleIntensityRanged(keys=['image'], a_min=-125, a_max=275, b_min=0.0, b_max=1.0, clip=True),
    CropForegroundd(keys=['image', 'seg'], source_key='image'),
    SpatialPadd(keys=['image', 'seg'], spatial_size=CONFIG['spatial_size']),
    ToTensord(keys=['image', 'seg'])
])

## 4. Data processing

In [ ]:
class RSNADataset(CacheDataset):
    def __init__(self, data, transforms, cache_rate=1.0):
        super().__init__(data, transforms, cache_num=len(data), cache_rate=cache_rate)

train_ds = RSNADataset(train_list, train_transforms, cache_rate=CONFIG['cache_rate'])
val_ds = RSNADataset(val_list, val_transforms, cache_rate=CONFIG['cache_rate'])

## 5. DataLoader

In [ ]:
train_loader = DataLoader(train_ds, batch_size=CONFIG['batch_size'], shuffle=True,
                          num_workers=CONFIG['num_workers'], collate_fn=list_data_collate,
                          persistent_workers=False, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=1, shuffle=False,
                        num_workers=CONFIG['num_workers'],
                        persistent_workers=False, pin_memory=True)

# II. Unet 3D Architecture

## 1. Initialize the architecture

In [ ]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv3d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv3d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)

In [ ]:
class Down(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool3d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)

In [ ]:
class Up(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.up = nn.ConvTranspose3d(in_channels, in_channels // 2, kernel_size=2, stride=2)
        self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)

In [ ]:
class UNet3D(nn.Module):
    def __init__(self, in_channels=1, out_channels=6, init_features=16):
        super().__init__()
        features = init_features

        self.inc = DoubleConv(in_channels, features)
        self.down1 = Down(features, features * 2)
        self.down2 = Down(features * 2, features * 4)
        self.down3 = Down(features * 4, features * 8)
        self.down4 = Down(features * 8, features * 16)

        self.up1 = Up(features * 16, features * 8)
        self.up2 = Up(features * 8, features * 4)
        self.up3 = Up(features * 4, features * 2)
        self.up4 = Up(features * 2, features)

        self.outc = nn.Conv3d(features, out_channels, kernel_size=1)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)

        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)

        logits = self.outc(x)
        return logits

## 2. Initialize model

In [ ]:
from torchsummary import summary

model = UNet3D(
    in_channels=1,
    out_channels=CONFIG['num_classes'],
    init_features=CONFIG['init_features']
).to(device)

summary(model, (1, 96, 96, 96))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1       [-1, 16, 96, 96, 96]             448
       BatchNorm3d-2       [-1, 16, 96, 96, 96]              32
              ReLU-3       [-1, 16, 96, 96, 96]               0
            Conv3d-4       [-1, 16, 96, 96, 96]           6,928
       BatchNorm3d-5       [-1, 16, 96, 96, 96]              32
              ReLU-6       [-1, 16, 96, 96, 96]               0
        DoubleConv-7       [-1, 16, 96, 96, 96]               0
         MaxPool3d-8       [-1, 16, 48, 48, 48]               0
            Conv3d-9       [-1, 32, 48, 48, 48]          13,856
      BatchNorm3d-10       [-1, 32, 48, 48, 48]              64
             ReLU-11       [-1, 32, 48, 48, 48]               0
           Conv3d-12       [-1, 32, 48, 48, 48]          27,680
      BatchNorm3d-13       [-1, 32, 48, 48, 48]              64
             ReLU-14       [-1, 32, 48,

## 3. Trainer Class

In [ ]:
class UNetTrainer:
    def __init__(self, model, train_loader, val_loader, config, device,
                 checkpoint_dir='/kaggle/working', use_wandb=True):
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.config = config
        self.checkpoint_dir = checkpoint_dir
        self.use_wandb = use_wandb

        self.device = device
        self.model.to(self.device)

        # Loss and optimizer
        self.criterion = DiceLoss(to_onehot_y=True, softmax=True)
        self.optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])
        self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer, mode='max', patience=5, factor=0.5, verbose=True
        )
        self.dice_metric = DiceMetric(include_background=False, reduction='mean')

        # Training state
        self.start_epoch = 0
        self.best_dice = 0.0
        self.history = []
        self.wandb_run_id = None

        # Paths
        self.checkpoint_path = os.path.join(checkpoint_dir, 'checkpoint.pth')
        self.best_model_path = os.path.join(checkpoint_dir, 'best_model.pth')
        self.history_csv_path = os.path.join(checkpoint_dir, 'training_history.csv')

    def save_checkpoint(self, epoch, val_dice, is_best=False):
        """Save checkpoint"""
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'scheduler_state_dict': self.scheduler.state_dict(),
            'best_dice': self.best_dice,
            'history': self.history,
            'config': self.config,
            'wandb_run_id': self.wandb_run_id
        }

        # Save latest checkpoint
        torch.save(checkpoint, self.checkpoint_path)
        print(f"Checkpoint saved at epoch {epoch}")

        # Save best model
        if is_best:
            torch.save(checkpoint, self.best_model_path)
            print(f"Best model saved! Dice: {val_dice:.4f}")

    def load_checkpoint(self):
        """Load checkpoint if exists"""
        if os.path.exists(self.checkpoint_path):
            print(f"Loading checkpoint from {self.checkpoint_path}")
            checkpoint = torch.load(self.checkpoint_path, map_location=self.device)

            self.model.load_state_dict(checkpoint['model_state_dict'])
            self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            self.scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
            self.start_epoch = checkpoint['epoch'] + 1
            self.best_dice = checkpoint['best_dice']
            self.history = checkpoint['history']

            # Load wandb run ID if available
            if 'wandb_run_id' in checkpoint:
                self.wandb_run_id = checkpoint['wandb_run_id']

            print(f"Resumed from epoch {self.start_epoch}, best dice: {self.best_dice:.4f}")
            return True
        return False

    def save_history_csv(self):
        """Save training history to CSV"""
        if self.history:
            df = pd.DataFrame(self.history)
            df.to_csv(self.history_csv_path, index=False)
            print(f"History saved to {self.history_csv_path}")

    def train_epoch(self, epoch):
        """Train one epoch"""
        self.model.train()
        total_loss = 0

        pbar = tqdm(self.train_loader, desc=f"Epoch {epoch} [Train]")
        for batch in pbar:
            images = batch['image'].to(self.device)
            segs = batch['seg'].to(self.device).long()

            self.optimizer.zero_grad()
            outputs = self.model(images)
            loss = self.criterion(outputs, segs)
            loss.backward()
            self.optimizer.step()

            total_loss += loss.item()
            pbar.set_postfix({'loss': f'{loss.item():.4f}'})

        avg_loss = total_loss / len(self.train_loader)
        return avg_loss

    def validate(self, epoch):
        """Validate"""
        self.model.eval()
        self.dice_metric.reset()

        with torch.no_grad():
            pbar = tqdm(self.val_loader, desc=f"Epoch {epoch} [Val]")
            for batch in pbar:
                images = batch['image'].to(self.device)
                segs = batch['seg'].to(self.device)

                outputs = sliding_window_inference(
                    images, self.config['spatial_size'], 1, self.model, overlap=0.5
                )
                self.dice_metric(y_pred=outputs, y=segs)

        val_dice = self.dice_metric.aggregate().item()
        return val_dice

    def train(self, num_epochs=None):
        """Main training loop"""
        if num_epochs is None:
            num_epochs = self.config['num_epochs']

        # Try to resume from checkpoint
        resumed = self.load_checkpoint()

        # Initialize wandb
        if self.use_wandb:
            if resumed and self.wandb_run_id:
                # Resume existing wandb run
                wandb.init(
                    project="my-3D-Unet-segment-RSNA",
                    config=self.config,
                    resume="allow",
                    id=self.wandb_run_id,
                    name=f"unet3d_resumed_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
                )
                print(f"Resumed W&B run: {self.wandb_run_id}")
            else:
                # Create new wandb run
                run_name = f"unet3d_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
                wandb.init(
                    project="my-3D-Unet-segment-RSNA",
                    config=self.config,
                    name=run_name,
                    tags=["3d-unet", "organ-segmentation", "rsna"]
                )
                self.wandb_run_id = wandb.run.id
                print(f"Created new W&B run: {self.wandb_run_id}")

            # Watch model
            wandb.watch(self.model, log='all', log_freq=100)

        print(f"\nStarting training from epoch {self.start_epoch} to {num_epochs}")
        print(f"Device: {self.device}")
        print(f"Model parameters: {sum(p.numel() for p in self.model.parameters()) / 1e6:.2f}M")

        for epoch in range(self.start_epoch, num_epochs):
            # Train
            train_loss = self.train_epoch(epoch + 1)

            # Validate
            val_dice = self.validate(epoch + 1)

            # Scheduler step
            self.scheduler.step(val_dice)

            # Get current learning rate
            current_lr = self.optimizer.param_groups[0]['lr']

            # Log metrics
            metrics = {
                'epoch': epoch + 1,
                'train_loss': train_loss,
                'val_dice': val_dice,
                'learning_rate': current_lr,
                'best_dice': self.best_dice
            }

            self.history.append(metrics)

            # Print metrics
            print(f"\nEpoch {epoch+1}/{num_epochs}:")
            print(f"  Train Loss: {train_loss:.4f}")
            print(f"  Val Dice: {val_dice:.4f}")
            print(f"  LR: {current_lr:.6f}")
            print(f"  Best Dice: {self.best_dice:.4f}")

            # Log to wandb
            if self.use_wandb:
                wandb.log(metrics)

            # Check if best model
            is_best = val_dice > self.best_dice
            if is_best:
                self.best_dice = val_dice

            # Save checkpoint
            self.save_checkpoint(epoch, val_dice, is_best)

            # Save history CSV
            self.save_history_csv()

        print(f"\nTraining complete! Best Dice: {self.best_dice:.4f}")

        if self.use_wandb:
            wandb.finish()

## 4. Create trainer

In [ ]:
trainer = UNetTrainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    config=CONFIG,
    device=device,
    # checkpoint_dir='/kaggle/working'    # Load model checkpoint
    checkpoint_dir=output_dir,
    use_wandb=use_wandb
)

# III. Start training

In [ ]:
# Clean memory
import gc

gc.collect()
torch.cuda.empty_cache()

In [ ]:
gc.collect()
torch.cuda.empty_cache()

trainer.train(num_epochs=CONFIG['num_epochs'])

Created new W&B run: u1dfv01h

Starting training from epoch 0 to 20
Device: cpu
Model parameters: 5.65M


Epoch 1 [Train]:   5%|▍         | 8/164 [04:24<1:18:53, 30.34s/it, loss=0.9538]

# IV. Load Best Model for Inference

In [ ]:
best_checkpoint = torch.load(trainer.best_model_path, map_location=device)
model.load_state_dict(best_checkpoint['model_state_dict'])
print(f"Best model loaded from epoch {best_checkpoint['epoch']} with Dice: {best_checkpoint['best_dice']:.4f}")

# V. Evaluation & Visualization

In [ ]:
class SegmentationEvaluator:
    def __init__(self, model, val_loader, device, num_classes=6):
        self.model = model
        self.val_loader = val_loader
        self.device = device
        self.num_classes = num_classes
        self.class_names = ['Background', 'Liver', 'Spleen', 'Kidney_L', 'Kidney_R', 'Bowel']

    def compute_metrics(self):
        """Compute comprehensive metrics"""
        self.model.eval()

        metrics = {
            'dice': [],
            'iou': [],
            'precision': [],
            'recall': [],
            'specificity': [],
            'hausdorff': []
        }

        # Per-class metrics
        class_metrics = {cls: {metric: [] for metric in metrics.keys()}
                        for cls in range(1, self.num_classes)}  # Skip background

        print("Computing metrics on validation set...")
        with torch.no_grad():
            for batch in tqdm(self.val_loader):
                images = batch['image'].to(self.device)
                labels = batch['seg'].to(self.device)

                # Inference
                outputs = sliding_window_inference(
                    images, (96, 96, 96), 1, self.model, overlap=0.5
                )
                preds = torch.argmax(outputs, dim=1)

                # Convert to numpy
                pred_np = preds.cpu().numpy()
                label_np = labels.cpu().numpy()

                # Compute metrics per class
                for cls in range(1, self.num_classes):
                    pred_cls = (pred_np == cls).astype(np.float32)
                    label_cls = (label_np == cls).astype(np.float32)

                    # Skip if no ground truth for this class
                    if label_cls.sum() == 0:
                        continue

                    # Dice
                    intersection = (pred_cls * label_cls).sum()
                    dice = (2 * intersection) / (pred_cls.sum() + label_cls.sum() + 1e-8)
                    class_metrics[cls]['dice'].append(dice)

                    # IoU
                    union = pred_cls.sum() + label_cls.sum() - intersection
                    iou = intersection / (union + 1e-8)
                    class_metrics[cls]['iou'].append(iou)

                    # Precision, Recall, Specificity
                    tp = intersection
                    fp = pred_cls.sum() - intersection
                    fn = label_cls.sum() - i
                    tersection
                    tn = ((pred_cls == 0) & (label_cls == 0)).sum()

                    precision = tp / (tp + fp + 1e-8)
                    recall = tp / (tp + fn + 1e-8)
                    specificity = tn / (tn + fp + 1e-8)

                    class_metrics[cls]['precision'].append(precision)
                    class_metrics[cls]['recall'].append(recall)
                    class_metrics[cls]['specificity'].append(specificity)

                    # Hausdorff Distance (sample a subset for speed)
                    if intersection > 0:
                        try:
                            # Get surface points (simplified)
                            pred_points = np.argwhere(pred_cls[0] > 0)
                            label_points = np.argwhere(label_cls[0] > 0)

                            # Sample points if too many
                            if len(pred_points) > 1000:
                                pred_points = pred_points[np.random.choice(len(pred_points), 1000, replace=False)]
                            if len(label_points) > 1000:
                                label_points = label_points[np.random.choice(len(label_points), 1000, replace=False)]

                            hd = max(directed_hausdorff(pred_points, label_points)[0],
                                   directed_hausdorff(label_points, pred_points)[0])
                            class_metrics[cls]['hausdorff'].append(hd)
                        except:
                            pass

        # Aggregate metrics
        results = {}
        for cls in range(1, self.num_classes):
            results[self.class_names[cls]] = {
                metric: np.mean(values) if values else 0.0
                for metric, values in class_metrics[cls].items()
            }

        return results

    def plot_metrics(self, results):
        """Plot metrics comparison"""
        metrics_to_plot = ['dice', 'iou', 'precision', 'recall', 'specificity']

        fig, axes = plt.subplots(2, 3, figsize=(18, 10))
        axes = axes.flatten()

        for idx, metric in enumerate(metrics_to_plot):
            classes = list(results.keys())
            values = [results[cls][metric] for cls in classes]

            axes[idx].bar(classes, values, color='steelblue', alpha=0.8)
            axes[idx].set_title(f'{metric.upper()}', fontsize=14, fontweight='bold')
            axes[idx].set_ylabel('Score', fontsize=12)
            axes[idx].set_ylim([0, 1])
            axes[idx].grid(axis='y', alpha=0.3)
            axes[idx].tick_params(axis='x', rotation=45)

            # Add value labels
            for i, v in enumerate(values):
                axes[idx].text(i, v + 0.02, f'{v:.3f}', ha='center', fontsize=10)

        # Hausdorff Distance (separate scale)
        classes = list(results.keys())
        hd_values = [results[cls]['hausdorff'] for cls in classes]
        axes[5].bar(classes, hd_values, color='coral', alpha=0.8)
        axes[5].set_title('Hausdorff Distance (lower is better)', fontsize=14, fontweight='bold')
        axes[5].set_ylabel('Distance (voxels)', fontsize=12)
        axes[5].grid(axis='y', alpha=0.3)
        axes[5].tick_params(axis='x', rotation=45)

        for i, v in enumerate(hd_values):
            axes[5].text(i, v + max(hd_values)*0.02, f'{v:.2f}', ha='center', fontsize=10)

        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 'metrics_comparison.png'), dpi=150, bbox_inches='tight')
        plt.show()
        print(f"Metrics plot saved to {output_dir}/metrics_comparison.png")

    def visualize_predictions(self, num_samples=3):
        """Visualize overlay masks"""
        self.model.eval()

        # Color map for organs
        colors = ['black', 'red', 'green', 'blue', 'yellow', 'purple']
        cmap = ListedColormap(colors)

        samples_shown = 0

        with torch.no_grad():
            for batch in self.val_loader:
                if samples_shown >= num_samples:
                    break

                images = batch['image'].to(self.device)
                labels = batch['seg'].to(self.device)

                outputs = sliding_window_inference(
                    images, (96, 96, 96), 1, self.model, overlap=0.5
                )
                preds = torch.argmax(outputs, dim=1)

                # Convert to numpy
                img_np = images[0, 0].cpu().numpy()
                pred_np = preds[0].cpu().numpy()
                label_np = labels[0].cpu().numpy()

                # Select middle slices
                D, H, W = img_np.shape
                slices_to_show = [D//4, D//2, 3*D//4]

                fig, axes = plt.subplots(3, 3, figsize=(15, 15))

                for idx, slice_idx in enumerate(slices_to_show):
                    # Original image
                    axes[idx, 0].imshow(img_np[slice_idx], cmap='gray')
                    axes[idx, 0].set_title(f'Slice {slice_idx}: CT Image', fontsize=12)
                    axes[idx, 0].axis('off')

                    # Ground truth
                    axes[idx, 1].imshow(img_np[slice_idx], cmap='gray')
                    axes[idx, 1].imshow(label_np[slice_idx], cmap=cmap, alpha=0.5, vmin=0, vmax=5)
                    axes[idx, 1].set_title(f'Ground Truth', fontsize=12)
                    axes[idx, 1].axis('off')

                    # Prediction
                    axes[idx, 2].imshow(img_np[slice_idx], cmap='gray')
                    axes[idx, 2].imshow(pred_np[slice_idx], cmap=cmap, alpha=0.5, vmin=0, vmax=5)
                    axes[idx, 2].set_title(f'Prediction', fontsize=12)
                    axes[idx, 2].axis('off')

                # Add legend
                from matplotlib.patches import Patch
                legend_elements = [Patch(facecolor=colors[i], label=self.class_names[i])
                                 for i in range(1, len(colors))]
                fig.legend(handles=legend_elements, loc='lower center', ncol=5, fontsize=11)

                plt.tight_layout()
                plt.savefig(os.path.join(output_dir, f'overlay_sample_{samples_shown+1}.png'),
                           dpi=150, bbox_inches='tight')
                plt.show()

                samples_shown += 1

        print(f"Overlay visualizations saved to {output_dir}/overlay_sample_*.png")

    def plot_3d_visualization(self, sample_idx=0):
        """3D visualization of segmentation"""
        from mpl_toolkits.mplot3d import Axes3D

        self.model.eval()

        with torch.no_grad():
            for idx, batch in enumerate(self.val_loader):
                if idx != sample_idx:
                    continue

                images = batch['image'].to(self.device)
                labels = batch['seg'].to(self.device)

                outputs = sliding_window_inference(
                    images, (96, 96, 96), 1, self.model, overlap=0.5
                )
                preds = torch.argmax(outputs, dim=1)

                pred_np = preds[0].cpu().numpy()
                label_np = labels[0].cpu().numpy()

                # Create 3D visualization for each organ
                fig = plt.figure(figsize=(20, 10))

                for organ_idx in range(1, min(4, self.num_classes)):  # Show first 3 organs
                    # Ground truth
                    ax1 = fig.add_subplot(2, 3, organ_idx, projection='3d')
                    mask_gt = (label_np == organ_idx)
                    if mask_gt.sum() > 0:
                        z, y, x = np.where(mask_gt)
                        ax1.scatter(x[::10], y[::10], z[::10], c='red', marker='o',
                                  s=1, alpha=0.3)
                    ax1.set_title(f'{self.class_names[organ_idx]} - Ground Truth', fontsize=12)
                    ax1.set_xlabel('X')
                    ax1.set_ylabel('Y')
                    ax1.set_zlabel('Z')

                    # Prediction
                    ax2 = fig.add_subplot(2, 3, organ_idx + 3, projection='3d')
                    mask_pred = (pred_np == organ_idx)
                    if mask_pred.sum() > 0:
                        z, y, x = np.where(mask_pred)
                        ax2.scatter(x[::10], y[::10], z[::10], c='blue', marker='o',
                                  s=1, alpha=0.3)
                    ax2.set_title(f'{self.class_names[organ_idx]} - Prediction', fontsize=12)
                    ax2.set_xlabel('X')
                    ax2.set_ylabel('Y')
                    ax2.set_zlabel('Z')

                plt.tight_layout()
                plt.savefig(os.path.join(output_dir, '3d_visualization.png'),
                           dpi=150, bbox_inches='tight')
                plt.show()

                print(f"3D visualization saved to {output_dir}/3d_visualization.png")
                break

    def create_summary_report(self, results):
        """Create text summary report"""
        report_path = os.path.join(output_dir, 'evaluation_report.txt')

        with open(report_path, 'w') as f:
            f.write("="*60 + "\n")
            f.write("SEGMENTATION EVALUATION REPORT\n")
            f.write("="*60 + "\n\n")

            for cls_name, metrics in results.items():
                f.write(f"\n{cls_name}:\n")
                f.write("-" * 40 + "\n")
                f.write(f"  Dice Coefficient:  {metrics['dice']:.4f}\n")
                f.write(f"  IoU:              {metrics['iou']:.4f}\n")
                f.write(f"  Precision:        {metrics['precision']:.4f}\n")
                f.write(f"  Recall:           {metrics['recall']:.4f}\n")
                f.write(f"  Specificity:      {metrics['specificity']:.4f}\n")
                f.write(f"  Hausdorff Dist:   {metrics['hausdorff']:.2f} voxels\n")

            # Overall average (excluding background)
            f.write("\n" + "-"*60 + "\n")
            f.write("OVERALL AVERAGES:\n")
            f.write("-"*60 + "\n")

            for metric in ['dice', 'iou', 'precision', 'recall', 'specificity']:
                avg = np.mean([results[cls][metric] for cls in results.keys()])
                f.write(f"  Average {metric.upper()}: {avg:.4f}\n")

        print(f"\nEvaluation report saved to {report_path}")

        # Print to console
        with open(report_path, 'r') as f:
            print(f.read())

In [ ]:
evaluator = SegmentationEvaluator(model, val_loader, device, num_classes=CONFIG['num_classes'])

## 1. Compute all metrics

In [ ]:
print("Computing metrics")
results = evaluator.compute_metrics()

## 2. Plot metrics comparison

In [ ]:
print("Plotting metrics comparison")
evaluator.plot_metrics(results)

## 3. Visualize overlay masks

In [ ]:
print("Creating overlay visualizations")
evaluator.visualize_predictions(num_samples=3)

## 4. 3D visualization

In [ ]:
evaluator.plot_3d_visualization(sample_idx=0)

## 5. Create summary report

In [ ]:
evaluator.create_summary_report(results)